In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.callbacks import EarlyStopping

C:\Users\joyho\AppData\Roaming\Python\Python38\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\joyho\AppData\Roaming\Python\Python38\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\joyho\AppData\Roaming\Python\Python38\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
path = "./spam.csv"
df = pd.read_csv(path,delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
X = df.v2 # 获取邮件内容
Y = df.v1 # 获取邮件分类
# 将标签转换为数值
Y = np.array([0 if i=='ham' else 1 for i in Y])

In [5]:
# 训练集测试集分割
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [6]:
# 分词，所有文本只考虑出现频率最多的 1000 个单词。
tok = tf.keras.preprocessing.text.Tokenizer(num_words=1000)
tok.fit_on_texts(X_train)
# 单词转数值序列
sequences = tok.texts_to_sequences(X_train)
# 设置最多只读邮件前150个单词
sequences_matrix = tf.keras.preprocessing.sequence.pad_sequences(sequences,maxlen=150)

In [7]:
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Embedding(1000, 50, input_length=150))
model.add(LSTM(64))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 96,337
Trainable params: 96,337
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])

In [9]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.00001)])

Epoch 1/10
28/28 [==============================] - 3s 24ms/step - loss: 0.4591 - accuracy: 0.8468 - val_loss: 0.3060 - val_accuracy: 0.8655
Epoch 2/10
28/28 [==============================] - 0s 11ms/step - loss: 0.1823 - accuracy: 0.9313 - val_loss: 0.0748 - val_accuracy: 0.9821


In [11]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = tf.keras.preprocessing.sequence.pad_sequences(test_sequences,maxlen=150)
accr = model.evaluate(test_sequences_matrix,Y_test)
print('测试集\n  损失: {:0.3f}\n  正确率: {:0.3f}'.format(accr[0],accr[1]))

35/35 [==============================] - 0s 8ms/step - loss: 0.0594 - accuracy: 0.9830
测试集
  损失: 0.059
  正确率: 0.983
